# XAI Analytics Notebook

In [1]:
import os
import xai
import logging as log 
import ipywidgets as widgets
import util.commons
import eli5
import random

from xai import imbalance_plot
from datetime import date
from util.commons import *
from util.ui import *
from util.model import *
from util.split import *
from util.dataset import *
from ipywidgets import interact, interact_manual, interactive, interactive_output
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Label
from IPython.display import clear_output

c:\projects\private\python\github\xai-analytics\xai-analytics\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\projects\private\python\github\xai-analytics\xai-analytics\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Select the dataset that you would like to invastigate later with the help of the available interpretability tools.

In [2]:
dataset_select_label = Label(layout=Layout(width='auto', height='auto'), value='Select a dataset (you can specify custom one by selecting other):')
dataset_select_dropdown = widgets.Dropdown(options=[m.name for m in Datasets], value=None, layout=Layout(width='200px', height='auto'))
name_url_text_label = Label(value='Provide dataset name and URL:')
name_text = widgets.Text(description='Name: ', placeholder='e.g. Car Evaluation Data Set', disabled=True)
url_text = widgets.Text(description='URL: ', placeholder='e.g. https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', disabled=True)
dataset_select_button = widgets.Button(description='Download dataset', layout=Layout(width='300px', height='auto'), style=ButtonStyle(button_color='green'), tooltip='Click me', icon='download', disabled=True)
dataset_select_output = widgets.Output()

display(dataset_select_label,
        dataset_select_dropdown,
        name_url_text_label,
        name_text,
        url_text,
        dataset_select_button,
        dataset_select_output)

def on_value_change_dataset_select_dropdown(change):
    dataset_select_output.clear_output()
    global dataset
    new_value = str(change['new'])
    if new_value == 'other':
        name_text.disabled=False
        url_text.disabled=False
        dataset_select_button.disabled=False
    else:
        name_text.disabled=True
        url_text.disabled=True
        dataset_select_button.disabled=True
        dataset, msg = get_dataset(new_value)
        with dataset_select_output:
            display(msg)
            
def on_click_dataset_select_button(self):
    dataset_select_output.clear_output()
    global dataset
    name = str(name_text.value)
    url = str(url_text.value)
    dataset, msg = get_dataset(name, url)
    with dataset_select_output:
        display(msg)

dataset_select_dropdown.observe(on_value_change_dataset_select_dropdown, names='value')
dataset_select_button.on_click(on_click_dataset_select_button)

# Delete lines after this comment
dataset, msg = get_dataset('census')

Label(value='Select a dataset (you can specify custom one by selecting other):', layout=Layout(height='auto', …

Dropdown(layout=Layout(height='auto', width='200px'), options=('anes96', 'cancer', 'ccard', 'china_smoking', '…

Label(value='Provide dataset name and URL:')

Text(value='', description='Name: ', disabled=True, placeholder='e.g. Car Evaluation Data Set')

Text(value='', description='URL: ', disabled=True, placeholder='e.g. https://archive.ics.uci.edu/ml/machine-le…

Button(description='Download dataset', disabled=True, icon='download', layout=Layout(height='auto', width='300…

Output()

26-Apr-20 16:57:50 - Dataset 'census (Adult census dataset)' loaded successfully. For further information about this dataset please visit: https://ethicalml.github.io/xai/index.html?highlight=load_census#xai.data.load_census
26-Apr-20 16:57:50 - 
   age          workclass   education  education-num       marital-status  \
0   39          State-gov   Bachelors             13        Never-married   
1   50   Self-emp-not-inc   Bachelors             13   Married-civ-spouse   
2   38            Private     HS-grad              9             Divorced   
3   53            Private        11th              7   Married-civ-spouse   
4   28            Private   Bachelors             13   Married-civ-spouse   

           occupation    relationship ethnicity   gender  capital-gain  \
0        Adm-clerical   Not-in-family     White     Male          2174   
1     Exec-managerial         Husband     White     Male             0   
2   Handlers-cleaners   Not-in-family     White     Male            

In [3]:
from pandas.api.types import is_numeric_dtype, is_string_dtype

strip_column_select_label = Label(layout=Layout(width='auto', height='auto'), value='Strip a column from the dataset:')
strip_column_select_dropdown = widgets.Dropdown(options=list(dataset.df.columns), value=None, layout=Layout(width='220px', height='auto'))
strip_button = widgets.Button(disabled=False, style=ButtonStyle(button_color='yellow'), tooltip='Strips everything except the selected value.', icon='bolt', layout=Layout(width='max-content', height='auto'))
strip_column_output = widgets.Output()
strip_column_output_inner = widgets.Output()

# back up initial dataset
df_backup = dataset.df

#defaults for the cell
eq_value = '='
df_stripped = None

def on_value_change_strip_column_select_dropdown(change):
    strip_column_output.clear_output()
    strip_column_output_inner.clear_output()
    strip_button.description = ''
    new_value = str(change['new'])
    if is_numeric_dtype(dataset.df[new_value]):
        eq_radio = init_strip_eq_radio(on_value_change_eq_radio)
        min_val, max_val, step=calculate_slider_properties(dataset.df[new_value].unique())
        value_slider = init_strip_value_slider(on_value_change_value_slider, min_val, max_val, step)
        with strip_column_output:
            display(eq_radio, value_slider, strip_button, strip_column_output_inner)
    elif is_string_dtype(dataset.df[new_value]):
        value_select_dropdown = init_strip_value_select_dropdown(on_value_change_value_select_dropdown, list(dataset.df[new_value].unique()))
        with strip_column_output:
            display(value_select_dropdown, strip_button, strip_column_output_inner)
            
def on_value_change_value_select_dropdown(change):
    strip_column_output_inner.clear_output()
    global df_stripped
    new_value = str(change['new'])
    with strip_column_output_inner:
        strip_button.description='{} \'{}\''.format(strip_column_select_dropdown.value, new_value)
        df_stripped = get_stripped_df(dataset.df, strip_column_select_dropdown.value, new_value)
        display(df_stripped)

def on_value_change_value_slider(change):
    strip_column_output_inner.clear_output()
    global df_stripped
    new_value = float(str(change['new']))    
    with strip_column_output_inner:
        strip_button.description='{} {} \'{}\''.format(strip_column_select_dropdown.value, eq_value, new_value)
        df_stripped = get_stripped_df(dataset.df, strip_column_select_dropdown.value, new_value, eq_value)
        display(df_stripped)

def on_value_change_eq_radio(change):
    global eq_value
    eq_value = str(change['new'])
            
def on_click_strip_button(self):
    strip_column_output.clear_output()
    global stack_label, df_stripped
    dataset.df = df_stripped
    dataset.df.reset_index(drop=True, inplace=True)
    stack_label.value = stack_label.value + strip_button.description + ', ' 
    with strip_column_output:
        display('{} selected successfully.'.format(strip_button.description), dataset.df)
    
def on_click_reset_button(self):
    strip_column_output.clear_output()
    dataset.df = df_backup
    dataset.df.reset_index(drop=True, inplace=True)
    global stack_label
    stack_label.value = 'Stripped columns for the dataset: '
    with strip_column_output:
        display('Dataset restored to its initial state.', dataset.df)

hbox = generate_reset_strip_hbox(on_click_reset_button)
stack_label = get_reset_strip_hbox_label(hbox)
    
strip_column_select_dropdown.observe(on_value_change_strip_column_select_dropdown, names='value')
strip_button.on_click(on_click_strip_button)

display(hbox, strip_column_select_label, strip_column_select_dropdown, strip_column_output)

Label(value='Strip a column from the dataset:', layout=Layout(height='auto', width='auto'))

Dropdown(layout=Layout(height='auto', width='220px'), options=('age', 'workclass', 'education', 'education-num…

Output()

Analyze dataset with the XAI toolset.

In [4]:
show_imbalance_selectmultiple = widgets.SelectMultiple(options=list(dataset.df.columns), rows=len(list(dataset.df.columns)) if len(list(dataset.df.columns)) <= 20 else 20, layout=Layout(width='auto', height='auto'))
show_imbalance_button = widgets.Button(description='Show imbalances', layout=Layout(width='auto', height='auto'), button_style='info', tooltip='Click me', icon='cubes')
correlations_matrix_button = Button(description='Correlations as a hierarchical dendogram', tooltip='Click me', icon='sitemap', layout=Layout(width='auto', height='auto'), disabled=False, style=ButtonStyle(button_color='darkseagreen'))
correlations_dendogram_button = Button(description='Correlations as a matrix', tooltip='Click me', icon='th-large', layout=Layout(width='auto', height='auto'), disabled=False, style=ButtonStyle(button_color='orange'))
show_imbalance_output = widgets.Output()

def on_click_show_imbalance_button(self):
    show_imbalance_output.clear_output()
    features_to_analyze = list(show_imbalance_selectmultiple.value)
    with show_imbalance_output:
        xai.imbalance_plot(dataset.df, *features_to_analyze)
        
def on_click_correlations_matrix_button(self):
    show_imbalance_output.clear_output()
    with show_imbalance_output:
        display(xai.correlations(dataset.df, include_categorical=True, plot_type="matrix"))
        
def on_click_correlations_dendogram_button(self):
    show_imbalance_output.clear_output()
    with show_imbalance_output:
        display(xai.correlations(dataset.df, include_categorical=True))

show_imbalance_button.on_click(on_click_show_imbalance_button)
correlations_matrix_button.on_click(on_click_correlations_matrix_button)
correlations_dendogram_button.on_click(on_click_correlations_dendogram_button)

grid_box = generate_analyze_grid(show_imbalance_selectmultiple,
                                 show_imbalance_button,
                                 correlations_dendogram_button,
                                 correlations_matrix_button)

display(grid_box, show_imbalance_output)

GridBox(children=(Label(value='Features to analyze: ', layout=Layout(height='auto', width='auto')), Button(dis…

Output()

Select a target and the number of models to be trained, analyzed and compared.

In [5]:
target_dropdown = widgets.Dropdown(options=list(dataset.df.columns), value=None, disabled=False)
target_select_button = widgets.Button(description='Select target', disabled=False, button_style='success', tooltip='Click me', icon='mouse-pointer')
target_output = widgets.Output()

display(target_dropdown, target_select_button, target_output)

def on_value_change_target_dropdown(change):
    target_output.clear_output()
    df_target, msg = show_target(dataset.df, change['new'])
    with target_output:
        display(df_target)

def on_click_target_select_button(slef):
    target_output.clear_output()
    global df_X, df_y
    df_X, df_y, msg = split_feature_target(dataset.df, target_dropdown.value)
    with target_output:
        display(msg)

target_dropdown.observe(on_value_change_target_dropdown, names='value')
target_select_button.on_click(on_click_target_select_button)

# Delete lines after this comment
df_X, df_y, msg = split_feature_target(dataset.df, "loan")

Dropdown(options=('age', 'workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relations…

Button(button_style='success', description='Select target', icon='mouse-pointer', style=ButtonStyle(), tooltip…

Output()

26-Apr-20 16:57:51 - Target 'loan' selected successfully.


Train a model by selecting its properties. _Hover the mouse over the description of a property in order to get more information about it._

In [6]:
models_label = Label(layout=Layout(width='auto', height='auto'), value='Choose the number of models to be used: ')
models_slider = widgets.IntSlider(value=1, min=1, max=8, step=1, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
models_output = widgets.Output()

def draw_grid():
    display(generate_model_grid(
        df_X,
        number_of_models,
        models,
        on_click_feature_exclude_button=on_click_feature_exclude_button,
        on_value_change_split_type_dropdown=on_value_change_split_type_dropdown,
        on_click_model_train_button=on_click_model_train_button))

def on_value_change_models_slider(change):
    models_output.clear_output()
    global number_of_models, models
    number_of_models = change['new']
    models, _ = fill_empty_models(df_X, df_y, number_of_models)
    with models_output:
        draw_grid()

models_slider.observe(on_value_change_models_slider, names='value')
display(models_label, models_slider, models_output)

def on_value_change_split_type_dropdown(change):
    model = get_model_by_split_type_dd(models, change['owner'])
    _ = change_cross_columns_status(model, change['new'])

def on_click_feature_exclude_button(self):
    models_output.clear_output()
    model = get_model_by_remove_features_button(models, self)
    msg = remove_model_features(model)
    with models_output:
        draw_grid()

def on_click_model_train_button(self):
    model = get_model_by_train_model_button(models, self)
    msg = fill_model(model)
    with models_output:
        display(msg)

# initially show only one model
with models_output:
    number_of_models = 1
    models, _ = fill_empty_models(df_X, df_y, number_of_models)
    draw_grid()

Label(value='Choose the number of models to be used: ', layout=Layout(height='auto', width='auto'))

IntSlider(value=1, continuous_update=False, max=8, min=1)

Output()

In [ ]:
kur
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# Reference for customizing matplotlib: https://matplotlib.org/users/style_sheets.html
plt.style.use('ggplot')

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

data = load_breast_cancer()
# Description of the data
print(data.DESCR)

pd.DataFrame(data.target_names)

X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
eclf = eclf.fit(X_train, y_train)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)

models = {'lr':clf1, 
          'rf':clf2, 
          'gnb':clf3, 
          'ensemble':eclf}

# Ensemble Classifier does not have feature importance enabled by default
f, axes = plt.subplots(2, 2, figsize = (26, 18))

ax_dict = {
    'lr':axes[0][0],
    'rf':axes[1][0],
    'gnb':axes[0][1],
    'ensemble':axes[1][1]
}

interpreter = Interpretation(X_test, feature_names=data.feature_names)

for model_key in models:
    pyint_model = InMemoryModel(models[model_key].predict_proba, examples=X_test)
    ax = ax_dict[model_key]
    interpreter.feature_importance.plot_feature_importance(pyint_model, ascending=True, ax=ax)
    ax.set_title(model_key)

26-Apr-20 16:57:51 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
26-Apr-20 16:57:51 - Loaded backend agg version unknown.


.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

2020-04-26 16:57:52,749 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


In [ ]:
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

for model in models:
    n, c = divide_features(model.X_test)
    fts = get_all_features(model.model, n, c)
    # print(fts)
    # interpreter = Interpretation(model.X, feature_names=(n+c))
    interpreter = Interpretation(training_data=model.X_test, training_labels=model.y_test, feature_names=list(n+c))
    im_model = InMemoryModel(model.model.predict_proba, examples=model.X_test, target_names=['$50K or less', 'More than $50K'])
    # interpreter.feature_importance.plot_feature_importance(pyint_model, ascending=True)
    plots = interpreter.feature_importance.plot_feature_importance(im_model, ascending=False)

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# Reference for customizing plots : http://matplotlib.org/users/customizing.html
# print(plt.style.available)
from sklearn.metrics import f1_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
print("Classes {}".format(np.unique(y)))

models = {'gb':GradientBoostingClassifier(), 
          'mlp':MLPClassifier(), 
          'knn':KNeighborsClassifier(), 
          'reg':LogisticRegression()}

for model_key in models:
    model = models[model_key]
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds)
    print("F1 for {0}: {1}".format(model_key, f1))
    
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

interpreter = Interpretation(X_test, feature_names=data.feature_names)
model = InMemoryModel(models['knn'].predict_proba, examples=X_train)
plots = interpreter.feature_importance.plot_feature_importance(model, ascending = False, progress_bar=False)

Classes [0 1]
F1 for gb: 0.9517241379310345
F1 for mlp: 0.9305555555555556
F1 for knn: 0.9090909090909091
F1 for reg: 0.9090909090909091


2020-04-28 18:32:22,092 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False


Interpret model globally with eli5.

In [ ]:
"""
generate_global_interpretation_grid(models)
for model in models:
    num_features, cat_features = divide_features(model.X)
    log.info("Global explanation of {}.".format(model.name))
    display(interpret_model(model.model, num_features, cat_features))
"""

Interpret a single example locally with LIME.

In [ ]:
"""
for i in range(3):
    rand = random.randrange(100)
    for model in models:
        log.info("Local explanation for {}.".format(model.name))
        explanation = explain_single_instance(model.model, model.X, model.y, rand)
        explanation.show_in_notebook(show_table=True, show_all=True)
"""

...